# Importing Essential Libraries

In [ ]:
import sys, os

import numpy as np
import pandas as pd
import geopandas as gpd

from tqdm import tqdm

In [ ]:
import bokeh.palettes as bokeh_palettes
import bokeh.models as bokeh_models

In [ ]:
from st_visualizer import st_visualizer
import express as viz_express
import geom_helper as viz_helper
import callbacks

# Defining Global Variables

In [ ]:
NOTEBOOK_URL='http://<NOTEBOOK_IP_ADDRESS>:<NOTEBOOK_PORT>'

 * # Use-Case #1: Using **CategoricalColorMapper** for a **Single Feature**.
   * ## NOTE: The Categorical Feature must be in String form

# Loading Polygon Dataset

In [ ]:
world_airports = pd.read_csv('data/csv/world_airports.csv')
world_airports = viz_helper.getGeoDataFrame_v2(world_airports, coordinate_columns=['Longitude', 'Latitude'], crs='epsg:4326')

''' For webmercator, when latitudes tend to 90 deg, northing tend to infinity, so using 89.9999 is not the solution. 
    The recommended area of use of EPSG:3857 is for latitudes between -85 and 85 degrees.
    Thus, we get the locations that reside anywhere but the poles. '''

world_airports = world_airports.loc[world_airports.Latitude.between(-85, 85)] 

In [ ]:
st_viz = st_visualizer(limit=500)
st_viz.set_data(world_airports, sp_columns=['Longitude', 'Latitude'])

st_viz.create_canvas(title=f'Prototype Plot', sizing_mode='scale_width', plot_height=540, tools="pan,box_zoom,lasso_select,wheel_zoom,previewsave,reset")
st_viz.add_map_tile('CARTODBPOSITRON')


st_viz.add_categorical_colormap('Category20b', 'City_Country')
plgn = st_viz.add_glyph(glyph_type='circle', fill_color=st_viz.cmap, line_color=st_viz.cmap, alpha=1, legend_group='City_Country')


tooltips = [('Name','@Name'), ('City Country','@City_Country'), ('IATA','@IATA'), ('Location','(@Longitude, @Latitude, @Altitude)'), ('Timezone', '@Timezone.1')]
st_viz.add_hover_tooltips(tooltips)
st_viz.add_lasso_select()

st_viz.figure.legend.location = "top_left"
st_viz.figure.legend.click_policy = "mute"
st_viz.figure.toolbar.active_scroll = st_viz.figure.select_one(bokeh_models.WheelZoomTool)

st_viz.show_figures(notebook=True, notebook_url=NOTEBOOK_URL)

## For Instance, observe that in order to effectively vizualize the - integer - categorigal feature ``mmsi``, its values had to be casted into String format prior to Mapping.

In [ ]:
st_viz = st_visualizer(limit=500)

st_viz.get_data_csv(filepath='./data/csv/ais_brest_2015-2016.csv', nrows=30000)
st_viz.data.loc[:, 'mmsi'] = st_viz.data.mmsi.astype(str)

st_viz.create_canvas(title=f'Prototype Plot', sizing_mode='scale_width', plot_width=540, plot_height=540, tools="pan,box_zoom,lasso_select,wheel_zoom,previewsave,reset")
st_viz.add_map_tile('CARTODBPOSITRON')

st_viz.add_categorical_colormap(palette=bokeh_palettes.Category10_3, categorical_name='mmsi')
circ = st_viz.add_glyph(glyph_type='circle', size=10, color=st_viz.cmap, alpha=0.8, fill_alpha=0.7, muted_alpha=0, legend_group=f'mmsi')

tooltips = [('Vessel ID','@mmsi'), ('Timestamp','@ts'), ('Speed (knots)','@speed'),('Course over Ground (degrees)','@course'), ('Heading (degrees)','@heading'), ('Coordinates','(@lon, @lat)')]
st_viz.add_hover_tooltips(tooltips)
st_viz.add_lasso_select()

st_viz.figure.legend.location = "top_left"
st_viz.figure.legend.click_policy = "mute"
st_viz.figure.toolbar.active_scroll = st_viz.figure.select_one(bokeh_models.WheelZoomTool)

st_viz.show_figures(notebook=True, notebook_url=NOTEBOOK_URL)

# In order to colormap numeric features like ``speed``, use LinearColorMapper

In [ ]:
st_viz = st_visualizer(limit=500)
st_viz.get_data_csv(filepath='./data/csv/ais_brest_2015-2016.csv', nrows=30000)

st_viz.create_canvas(title=f'Prototype Plot', sizing_mode='scale_width', plot_height=540, tools="pan,box_zoom,lasso_select,wheel_zoom,previewsave,reset")
st_viz.add_map_tile('CARTODBPOSITRON')


st_viz.add_numerical_colormap('Viridis256', 'speed', colorbar=True, cb_orientation='vertical', cb_location='right', label_standoff=12, border_line_color=None, location=(0,0))
circ = st_viz.add_glyph(glyph_type='circle', size=10, color=st_viz.cmap, alpha=0.8, fill_alpha=0.7, muted_alpha=0, legend_label=f'Speed Heatmap')


tooltips = [('Vessel ID','@mmsi'), ('Timestamp','@ts'), ('Speed (knots)','@speed'),('Course over Ground (degrees)','@course'), ('Heading (degrees)','@heading'), ('Coordinates','(@lon, @lat)')]
st_viz.add_hover_tooltips(tooltips)
st_viz.add_lasso_select()

st_viz.figure.legend.location = "top_left"
st_viz.figure.legend.click_policy = "mute"
st_viz.figure.toolbar.active_scroll = st_viz.figure.select_one(bokeh_models.WheelZoomTool)

st_viz.show_figures(notebook=True, notebook_url=NOTEBOOK_URL)

# Testing Colormaps & Filters Interaction

* #### Numerical Colormap Example

In [ ]:
st_viz = st_visualizer(limit=500)
st_viz.get_data_csv(filepath='./data/csv/ais_brest_2015-2016.csv', nrows=30000)

st_viz.create_canvas(title=f'Prototype Plot', sizing_mode='scale_width', plot_height=540, tools="pan,box_zoom,lasso_select,wheel_zoom,previewsave,reset")
st_viz.add_map_tile('CARTODBPOSITRON')


st_viz.add_numerical_colormap('Viridis256', 'speed', colorbar=True, cb_orientation='vertical', cb_location='right', label_standoff=12, border_line_color=None, location=(0,0))
st_viz.add_glyph(glyph_type='circle', size=10, color=st_viz.cmap, alpha=0.8, fill_alpha=0.7, muted_alpha=0, legend_label=f'GPS Locations (Speed Heatmap)')
st_viz.add_temporal_filter(callback_policy='value')


tooltips = [('Vessel ID','@mmsi'), ('Timestamp','@ts'), ('Speed (knots)','@speed'),('Course over Ground (degrees)','@course'), ('Heading (degrees)','@heading'), ('Coordinates','(@lon, @lat)')]
st_viz.add_hover_tooltips(tooltips)
st_viz.add_lasso_select()

st_viz.figure.legend.location = "top_left"
st_viz.figure.legend.click_policy = "mute"
st_viz.figure.toolbar.active_scroll = st_viz.figure.select_one(bokeh_models.WheelZoomTool)

st_viz.show_figures(notebook=True, notebook_url=NOTEBOOK_URL)

* #### Categorical Colormap Example

In [ ]:
st_viz = st_visualizer(limit=500)
st_viz.get_data_csv(filepath='./data/csv/ais_brest_2015-2016.csv', nrows=50000)

st_viz.data.loc[:, 'mmsi'] = st_viz.data.mmsi.astype(str)
# st_viz.data = st_viz.data.loc[st_viz.data.mmsi.isin(st_viz.data.mmsi.unique()[:20])]

st_viz.create_canvas(title=f'Prototype Plot', sizing_mode='scale_width', plot_height=540, tools="pan,box_zoom,lasso_select,wheel_zoom,previewsave,reset")
st_viz.add_map_tile('CARTODBPOSITRON')


st_viz.add_categorical_colormap(palette=bokeh_palettes.Category20c_20, categorical_name='mmsi')
st_viz.add_glyph(glyph_type='circle', size=10, color=st_viz.cmap, alpha=0.8, fill_alpha=0.7, muted_alpha=0, legend_field=f'mmsi')
st_viz.add_temporal_filter(callback_policy='value', step_ms=1000)

tooltips = [('Vessel ID','@mmsi'), ('Timestamp','@ts'), ('Speed (knots)','@speed'),('Course over Ground (degrees)','@course'), ('Heading (degrees)','@heading'), ('Coordinates','(@lon, @lat)')]
st_viz.add_hover_tooltips(tooltips)
st_viz.add_lasso_select()

st_viz.figure.legend.location = "top_left"
st_viz.figure.legend.click_policy = "mute"
st_viz.figure.toolbar.active_scroll = st_viz.figure.select_one(bokeh_models.WheelZoomTool)

st_viz.show_figures(notebook=True, notebook_url=NOTEBOOK_URL)